In [0]:
 from pyspark.sql.functions import *
 from pyspark.sql.types import *

# CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text("incremental_flag",'0')

In [0]:
incremental_flag = dbutils.widgets.get("incremental_flag")


# CREATING DIMENSION MODEL


In [0]:
# Selecting only the relevant columns from the silver table.
df_src = spark.sql('''select *  FROM parquet.`abfss://silver@sumitdatalake.dfs.core.windows.net/carsales`
        ''')

df_src.display()

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Day,Month,Year,BranchName,DealerName,Product_Name,model_category,RevPerUnit
BR0001,DLR0001,BMW-M1,13363978,2,DT00001,1,1,2017,AC Cars Motors,AC Cars Motors,BMW,BMW,26727956
BR0003,DLR0228,Hon-M218,17376468,3,DT00001,10,5,2017,AC Cars Motors,Deccan Motors,Honda,Hon,52129404
BR0004,DLR0208,Tat-M188,9664767,3,DT00002,12,1,2017,AC Cars Motors,Wiesmann Motors,Tata,Tat,28994301
BR0005,DLR0188,Hyu-M158,5525304,3,DT00002,16,9,2017,AC Cars Motors,Subaru Motors,Hyundai,Hyu,16575912
BR0006,DLR0168,Ren-M128,12971088,3,DT00003,20,5,2017,AC Cars Motors,Saab Motors,Renault,Ren,38913264
BR0008,DLR0128,Hon-M68,7321228,1,DT00004,28,4,2017,AC Cars Motors,Messerschmitt Motors,Honda,Hon,7321228
BR0009,DLR0108,Cad-M38,11379294,2,DT00004,31,12,2017,AC Cars Motors,Lexus Motors,Cadillac,Cad,22758588
BR0010,DLR0088,Mer-M8,11611234,2,DT00005,4,9,2017,AC Cars Motors,"IFA (including Trabant, Wartburg, Barkas) Motors",Mercedes-Benz,Mer,23222468
BR0011,DLR0002,BMW-M2,19979446,2,DT00005,2,1,2017,Acura Motors,Acura Motors,BMW,BMW,39958892
BR0011,DLR0069,Vol-M256,14181510,3,DT00006,9,5,2017,Acura Motors,Geo Motors,Volkswagen,Vol,42544530


# Fetch Relative Columns

In [0]:
%sql
select Distinct(Dealer_ID) as Dealer_ID, DealerName from parquet.`abfss://silver@sumitdatalake.dfs.core.windows.net/carsales`

Dealer_ID,DealerName
DLR0058,Fiat do Brasil Motors
DLR0107,Land Rover Motors
DLR0129,Mia Motors
DLR0111,Lotus Motors
DLR0085,Humber Motors
DLR0001,AC Cars Motors
DLR0218,Lagonda Motors
DLR0082,Honda Motors
DLR0063,Ford do Brasil Motors
DLR0193,Tazzari Motors


In [0]:
df_src = spark.sql('''select Distinct(Dealer_ID) as Dealer_ID, DealerName from parquet.`abfss://silver@sumitdatalake.dfs.core.windows.net/carsales`
        ''')

In [0]:
df_src.display()

Dealer_ID,DealerName
DLR0058,Fiat do Brasil Motors
DLR0107,Land Rover Motors
DLR0129,Mia Motors
DLR0111,Lotus Motors
DLR0085,Humber Motors
DLR0001,AC Cars Motors
DLR0218,Lagonda Motors
DLR0082,Honda Motors
DLR0063,Ford do Brasil Motors
DLR0193,Tazzari Motors


### dim_dealer Sink - Initial and Incremental (Just Bring the Schema if table NOT EXISTS


In [0]:
# This is for initial load and incremental loads
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer') :
    df_sink = spark.sql('''
    SELECT dim_dealer_key, Dealer_ID, DealerName FROM cars_catalog.gold.dim_dealer
    ''')

else :    
    df_sink = spark.sql('''
    SELECT 1 AS dim_dealer_key, Dealer_ID, DealerName FROM parquet.`abfss://silver@sumitdatalake.dfs.core.windows.net/carsales` WHERE 1 = 0
    ''')


### Filtering new records and old records 

In [0]:
df_filter = df_src.join(df_sink, df_src['Dealer_ID'] == df_sink['Dealer_ID'], 'left').select(df_src['Dealer_ID'], df_src['DealerName'], df_sink['dim_dealer_key'])

In [0]:
df_filter.display()

Dealer_ID,DealerName,dim_dealer_key
DLR0058,Fiat do Brasil Motors,null
DLR0107,Land Rover Motors,null
DLR0129,Mia Motors,null
DLR0111,Lotus Motors,null
DLR0085,Humber Motors,null
DLR0001,AC Cars Motors,null
DLR0218,Lagonda Motors,null
DLR0082,Honda Motors,null
DLR0063,Ford do Brasil Motors,null
DLR0193,Tazzari Motors,null


### df_filter_old


In [0]:
df_filter_old = df_filter.filter(col('dim_dealer_key').isNotNull())

In [0]:
df_filter_old.display()

Dealer_ID,DealerName,dim_dealer_key


### df_filter_new


In [0]:
df_filter_new = df_filter.filter(col('dim_dealer_key').isNull()).select(df_src['Dealer_ID'], df_src['DealerName'])

In [0]:
df_filter_new.display()

Dealer_ID,DealerName
DLR0058,Fiat do Brasil Motors
DLR0107,Land Rover Motors
DLR0129,Mia Motors
DLR0111,Lotus Motors
DLR0085,Humber Motors
DLR0001,AC Cars Motors
DLR0218,Lagonda Motors
DLR0082,Honda Motors
DLR0063,Ford do Brasil Motors
DLR0193,Tazzari Motors


## Create Surrogate Key

**Fetch the max surrogate key from existing table**

In [0]:
if(incremental_flag == '0'):
    max_value = 1 
else:
    max_value_df = spark.sql("select max(dim_branch_key) from cars_catalog.gold.dim_branch")
    max_value = max_value_df.collect()[0][0]+1


### Create Surrogate key column and ADD the max surrogate key

In [0]:
df_filter_new = df_filter_new.withColumn('dim_branch_key',max_value+monotonically_increasing_id())

In [0]:
df_filter_new.display()

Dealer_ID,DealerName,dim_branch_key
DLR0058,Fiat do Brasil Motors,1
DLR0107,Land Rover Motors,2
DLR0129,Mia Motors,3
DLR0111,Lotus Motors,4
DLR0085,Humber Motors,5
DLR0001,AC Cars Motors,6
DLR0218,Lagonda Motors,7
DLR0082,Honda Motors,8
DLR0063,Ford do Brasil Motors,9
DLR0193,Tazzari Motors,10


### Create Final DF - df_filter_old + df_filter_new

In [0]:
df_final = df_filter_old.union(df_filter_new)
df_final.display()

Dealer_ID,DealerName,dim_dealer_key
DLR0058,Fiat do Brasil Motors,1
DLR0107,Land Rover Motors,2
DLR0129,Mia Motors,3
DLR0111,Lotus Motors,4
DLR0085,Humber Motors,5
DLR0001,AC Cars Motors,6
DLR0218,Lagonda Motors,7
DLR0082,Honda Motors,8
DLR0063,Ford do Brasil Motors,9
DLR0193,Tazzari Motors,10


## SCD TYPE -1(UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
#Incremental Run
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
    delta_tbl = DeltaTable.forPath(spark,"abfss://gold@sumitdatalake.dfs.core.windows.net/dim_dealer")

    delta_tbl.alias("trg").merge(df_final.alias("src"), "trg.dim_dealer_key = src.dim_dealer_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()


#Initial RUN
else:
    df_final.write.format('delta')\
    .mode('overwrite')\
    .option("path","abfss://gold@sumitdatalake.dfs.core.windows.net/dim_dealer")\
    .saveAsTable('cars_catalog.gold.dim_dealer')

In [0]:
%sql
select * from cars_catalog.gold.dim_dealer

Dealer_ID,DealerName,dim_dealer_key
DLR0058,Fiat do Brasil Motors,1
DLR0107,Land Rover Motors,2
DLR0129,Mia Motors,3
DLR0111,Lotus Motors,4
DLR0085,Humber Motors,5
DLR0001,AC Cars Motors,6
DLR0218,Lagonda Motors,7
DLR0082,Honda Motors,8
DLR0063,Ford do Brasil Motors,9
DLR0193,Tazzari Motors,10
